In [18]:
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase Admin
cred = credentials.Certificate('path/to/your/serviceAccountKey.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

def gen_data(n):
    n_per_class = n // 2  # Ensure n is even for equal class distribution
    
    # Generate features
    labels = np.array([0] * n_per_class + [1] * n_per_class)
    hardness = np.concatenate([
        np.random.randint(3, 6, n_per_class),  # Softer for gold
        np.random.randint(1, 4, n_per_class)  # Harder for pyrite
    ])
    density = np.concatenate([
        np.random.uniform(4, 6, n_per_class),  # Closer to pyrite
        np.random.uniform(18, 20, n_per_class)  # Closer to gold
    ])
    conductivity = np.concatenate([
        np.random.randint(1, 4, n_per_class),  # Higher for gold
        np.random.randint(3, 6, n_per_class)  # Lower for pyrite
    ])
    shininess = np.concatenate([
        np.random.randint(3, 6, n_per_class),  # More shiny for gold
        np.random.randint(1, 3, n_per_class)  # Less shiny for pyrite
    ])
    # Direct generation of one-hot encoded 'shape' columns
    shapes = np.random.choice(['square', 'circle', 'rectangle'], size=n)
    shape_columns = pd.get_dummies(shapes, prefix='shape')

    # Direct generation of one-hot encoded 'color' columns
    colors = np.random.choice(['yellow', 'bronze yellow', 'silver yellow'], size=n)
    color_columns = pd.get_dummies(colors, prefix='color')
    
    # Create the new data DataFrame without 'shape' and 'color'
    new_data = pd.DataFrame({
        'label': labels,
        'hardness': hardness,
        'density': density,
        'conductivity': conductivity,
        'shininess': shininess,
    })
    
    # Concatenate with one-hot encoded columns
    new_data_encoded = pd.concat([new_data, shape_columns, color_columns], axis=1)

    # Convert DataFrame to dictionary for Firebase
    data_dict = new_data_encoded.to_dict(orient='records')

    # Push data to Firebase
    for record in data_dict:
        db.collection('your_collection_name').add(record)

# Example usage

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/serviceAccountKey.json'

In [17]:
# Example usage
gen_data(10)  # This will append 100 new rows to existing_data

existing_data.head(50)

,label,hardness,density,conductivity,shininess,shape_circle,shape_rectangle,shape_square,color_bronze yellow,color_silver yellow,color_yellow
0,0,5,4.767494,3,5,True,False,False,False,True,False
1,0,5,5.403695,2,3,True,False,False,True,False,False
2,0,4,4.030001,1,5,False,False,True,False,True,False
3,0,3,4.321032,1,5,True,False,False,False,False,True
4,0,4,5.255167,3,4,False,True,False,False,True,False
5,1,2,18.745585,4,1,False,False,True,False,False,True
6,1,3,18.243361,5,2,False,True,False,False,True,False
7,1,2,18.608224,5,2,False,False,True,False,False,True
8,1,3,19.966253,4,2,False,False,True,True,False,False
9,1,2,19.516223,3,2,True,False,False,False,True,False
